# Example of import_raster cmd using API of geonode_conabio and [kale](https://github.com/kubeflow-kale/kale) functionality

Reference: https://github.com/CONABIO/geonode/blob/master/notebooks/towards_module_python_to_ingest_programmatically_into_geonode/1_normalizing_and_reprojecting.ipynb

**Raster has small size**

In [1]:
import os

from pyproj import Proj
import rasterio
from docker import APIClient

from geonode_conabio.wrappers import reproj_and_write_one_band_raster
from geonode_conabio.utils_docker import import_layers_via_docker

Python-dotenv could not parse statement starting at line 1


In [2]:
path = "/shared_volume/land_cover_results"
filename = "raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_54_-38.tif"
filename = os.path.join(path, filename)
output_filename = "raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_54_-38_processed_for_geonode.tif"
output_filename = os.path.join(path, output_filename)
with rasterio.open(filename) as src:
    src_crs = src.crs.to_string()
    proj_crs = Proj(src_crs)
    if not proj_crs.crs.is_geographic:
        reproj_and_write_one_band_raster(src, output_filename,
                                         is_geographic=False)
    else:
        reproj_and_write_one_band_raster(src, output_filename)

In [3]:
print(filename)

/shared_volume/land_cover_results/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_54_-38.tif


In [4]:
print(output_filename)

/shared_volume/land_cover_results/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_54_-38_processed_for_geonode.tif


In [ ]:
region = "".join(["\"Chiapas, Mexico, North America, Latin America\""])
name = "\"Chiapas_lc_2017_landsat8_test\""
title = "\"Land cover Chiapas landsat8 2017 test using kale\""
abstract = "\"Test kale\""
kw = "\"Chiapas\""


c = APIClient(base_url='tcp://172.17.0.1:1111')
cmd = "".join(["python manage.py importlayers -v 3 -i -o ",
               "-n ", name, " ",
               "-t ", title, " ",
               "-a ", abstract, " ",
               "-k ", kw, " ",
               "-r ", region, " ",
               output_filename, " ",
               "--settings=geonode.local_settings"])
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)
ex_start = c.exec_start(exec_id=ex)

print(ex_start)
c.close()
os.remove(output_filename)

Maybe check this one:

https://github.com/CONABIO/geonode/issues/12

and use: 

https://github.com/CONABIO/geonode/blob/master/styles/madmex_31_classes.sld